# Setup
## Imports

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [ ]:
import constants
import config

In [ ]:
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

## Config

In [ ]:
base_path = "sweeps/"

In [ ]:
offline_color = "forestgreen"
measured_color = "darkorchid"
constant_color = "dodgerblue"

In [ ]:
success_angle = 0.99*np.pi

## Functions

In [ ]:
def draw_trajs(datas, idxs=None, plot_manipulator=True, progs=None, num_times=5,
              fig_height=2.1, cb_shrink=0.8, max_times=None):
    if idxs is None:
        sim_args = datas["sim_args"][0]
        datas = datas["out_data"]
    else:
        sim_args = datas["sim_args"][0]
        datas = [datas["out_data"][idx] for idx in idxs]
    w_L = sim_args['sim_params'].w_L
    h_L = sim_args['sim_params'].h_L
    r = sim_args['sim_params'].r
    if max_times is None:
        max_times = np.ones(len(datas))

    # Set to be arbitrary large/small numbers
    # IMPORTANT: x and y here refer to PLOT x and ys, not sim
    min_x = 10**9
    min_y = 10**9
    max_x = -10**9
    max_y = -10**9

    # Initialize plotting vars
    fig, axs = plt.subplots(1, len(datas), figsize=(6,fig_height), constrained_layout=True, sharex=True, sharey=True)
    if len(datas) == 1:
        axs = [axs]
    axs[0].set_ylabel("$z$ position")
    cmap = plt.get_cmap("viridis_r")
    
    for i, data_, in enumerate(datas):
        start_idx = data_["any_links_in_contact_idx"]
        start_time = data_["times"][start_idx]
        end_time = data_["times"][-1]
        time_range = end_time - start_time
#         color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(start_time,start_time +time_range*max_time))
        
        if progs is None:
            times_ = np.linspace(start_time, end_time, num_times)
        else:
            if type(progs[0]) is list:
                progs_ = np.array(progs[i])
            else:
                progs_ = np.array(progs)
            times_ = progs_*time_range + start_time
        ax = axs[i]

        # Draw rectangles over links
        for t in times_:
            c = cmap(((t-start_time)/time_range)/max_times[i])
            idx = np.argmax(data_["times"] >= t)

            for x, y, ang in zip(
                    data_["horizontal_paper_traces"][:,idx],
                    data_["vertical_paper_traces"][:,idx],
                    data_["angle_paper_traces"][:,idx]):

                x0 = x - np.cos(ang)*w_L/2
                y0 = y - np.sin(ang)*w_L/2

                rect = plt. Rectangle((x0, y0), w_L, h_L,
                    angle=ang*180/np.pi, color=c,)
                ax.add_patch(rect)

                # Compute bounds to be used
                min_x = min(x - w_L*1.1*0.5, min_x)
                min_y = min(y - w_L*1.1*0.5, min_y)
                max_x = max(x + w_L*1.1*0.5, max_x)
                max_y = max(y + w_L*1.1*0.5, max_y)

        # Add pedestal
        ## Get first pedestal location
        rect = plt.Rectangle(
            (data_["horizontal_paper_traces"][0,0]-w_L/2,
             data_["vertical_paper_traces"][0,0]-h_L/2),
            10,
            10,
            angle=-90,
            color=(0.2, 0.2, 0.2, 1.0)
        )
        ax.add_patch(rect)

        # Add manipulator
        if plot_manipulator:
            for t in times_:
                c = cmap((t-start_time)/time_range)
                idx = np.argmax(data_["times"] >= t)

                x = data_["horizontal_manipulator_trace"][idx]
                y = data_["vertical_manipulator_trace"][idx]

                min_x = min(x - r*1.1, min_x)
                min_y = min(y - r*1.1, min_y)
                max_x = max(x + r*1.1, max_x)
                max_y = max(y + r*1.1, max_y)

                circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
                ax.add_patch(circ)

        ax.set_xlabel("$y$ position")
        ax.set_aspect("equal")

    # Set lims (can do at the end because sharex and sharey are true)
    min_plot_x = min_x - (max_x-min_x)*0.1
    max_plot_x = max_x + (max_x-min_x)*0.1
    min_x_tick = np.floor(min_plot_x*10)/10
    max_x_tick = np.ceil(max_plot_x*10)/10
    xticks = np.arange(min_x_tick, max_x_tick+1e-5, 0.1)
    ax.set_xticks(xticks)

    min_plot_y = min_y - (max_y-min_y)*0.1
    max_plot_y = max_y + (max_y-min_y)*0.1
    min_y_tick = np.floor(min_plot_y*10)/10
    max_y_tick = np.ceil(max_plot_y*10)/10
    yticks = np.arange(min_y_tick, max_y_tick+1e-5, 0.1)
    ax.set_yticks(yticks)
    ax.set_ylim(min_plot_y, max_plot_y)
    plt.xlim(min_plot_x, max_plot_x)
    cb = fig.colorbar(cm.ScalarMappable(cmap="viridis_r"), shrink=cb_shrink)
    cb.set_label("Normalized time")
#     ticks = np.linspace(0, 100, 6)
#     cb.set_ticks(ticks/100)
#     cb.set_ticklabels([str(int(tick)) + "\%" for tick in ticks])

    return fig, axs, cb

# Intro sweeps

In [ ]:
data = np.load(base_path + "traces.npz", allow_pickle=True)

In [ ]:
plot_manipulator=True
data = np.load(base_path + "traces.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

fig, axs, cb = draw_trajs(data, plot_manipulator=plot_manipulator,
               fig_height=3)
titles = ["Offline setpoint\ntrajectory",
          r"Link feedback with" + "\n" +
          "constant $F_{Nff}$",
          "Link feedback with" + "\n" +
          "$F_{Nff}$ based on" + "\n" +
          "force measurements"]
for title, ax in zip(titles, axs):
    ax.set_title(title)
plt.suptitle("System behavior with various strategies")
# fig.savefig("traj_drawings.pgf", backend="pgf")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)
colors = [offline_color, constant_color, measured_color]
titles = ["Offline setpoint trajectory",
          r"Link feedback with constant $F_{Nff}$",
          "Link feedback with $F_{Nff}$ based on force measurements"]
for i, (data_, color, title) in enumerate(zip(data['out_data'], colors, titles)):
    start_idx = data_["any_links_in_contact_idx"]
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=color, label=title)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=color)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=color)
plt.legend(frameon=False)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.gca().set_aspect("equal")

fig.savefig("traj_plots.pgf", backend="pgf")
plt.show()

# Task parameters

## Stiffness

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__k_J.npz", allow_pickle=True),
    np.load(base_path + "offline__k_J.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__k_J.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Offline trajectory","Constant"]
colors = [measured_color,offline_color, constant_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"][:,0]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])/success_angle
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    plt.plot(x_axis, y_axis, linestyle='-', linewidth=2, label=label, zorder=i, color=color)
#     plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=color, zorder=i, s=3)
#     plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=color, zorder=i, s=3)
plt.legend(frameon=False)
plt.xlabel("Joint stiffness (Nm/rad)")
plt.ylabel("Angle about joint (radians)")
plt.autoscale(enable=True, axis='x', tight=True)
# plt.gcf().savefig("k_J_sweep.pgf", backend="pgf")
# plt.xlim(0,0.5)
plt.axhline(1, color='k', linestyle='--')
# plt.ylim(0,1.05)
plt.show()

Rerun with several impedances

In [ ]:
data = np.load(base_path + "measured_Fn__k_J.npz", allow_pickle=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, k_J) in enumerate(zip(data['out_data'][3:], data['sweep_vars'][3:,0])):
    c = color_map.to_rgba(k_J)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Link stiffness (Nm/rad)")
plt.gcf().savefig("k_J_plot.pgf", backend="pgf")
plt.show()

Possibly add more pictures of link behavior

## Link mass

### Sweep

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__m_L.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__m_L.npz", allow_pickle=True),
    np.load(base_path + "offline__m_L.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant", "Offline trajectory"]
colors = [measured_color, constant_color, offline_color]
fig, axs = plt.subplots(3, 1, figsize=(6,4), constrained_layout=True,sharex=True,sharey=True)

for i, (data, label, color, ax) in enumerate(zip(datas, labels, colors, axs)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])/success_angle
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    ax.plot(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    ax.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=color, zorder=i)
    ax.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=color, zorder=i)
    ax.set_ylabel("Progress")
# plt.legend(frameon=False)
plt.xlabel("Link mass (kg)")

plt.gcf().savefig("m_L_sweep.pgf", backend="pgf")
plt.show()

Explanation: In the middle, the stiffness is enough to pull it off; at the end, you can dump it and it will stay.

### Trajectories with measured $F_N$

In [ ]:
data = np.load(base_path + "measured_Fn__m_L.npz", allow_pickle=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

idxs = [1,-4,-1]
color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, k_J) in enumerate(zip(data['out_data'][idxs], data['sweep_vars'][idxs])):
    c = color_map.to_rgba(k_J)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Link stiffness (Nm/rad)")
plt.gcf().savefig("k_J_plot.pgf", backend="pgf")
plt.show()

Add a dashed line

In [ ]:
data = np.load(base_path + "measured_Fn__m_L.npz", allow_pickle=True)

idxs = [1,-4,-1]
end_times = [None,3.5,None]
fig, axs = plt.subplots(1, len(idxs), figsize=(6,2), constrained_layout=True,sharex=True,sharey=True)
ax.set_ylabel("$z$ position")
color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, k_J, ax,end_time) in enumerate(zip(data['out_data'][idxs], data['sweep_vars'][idxs], axs, end_times)):
    c = color_map.to_rgba(k_J)
    start_idx = data_["any_links_in_contact_idx"]
    if end_time is None:
        end_time = -1
    else:
        end_idx = np.argmax(data_["times"] > end_time)
    ax.scatter(data_["horizontal_paper_traces"][-1,start_idx:end_idx:100],
                data_["vertical_paper_traces"][-1,start_idx:end_idx:100],
                c=data_["times"][start_idx:end_idx:100],s=2)
    ax.set_aspect("equal")
    ax.set_xlabel("$y$ position")

# cb = fig.colorbar(color_map)
# cb.set_label("Link stiffness (Nm/rad)")
# plt.gcf().savefig("k_J_plot.pgf", backend="pgf")
plt.show()

In [ ]:
data = np.load(base_path + "measured_Fn__m_L.npz", allow_pickle=True)
draw_trajs(data, [2, -4,-1], plot_manipulator=False, progs=[
    [0,0.2,0.4,0.6,0.8,1],
    [0,0.12,0.17,0.2,0.25,0.27,0.3],
    [0.  , 0.1 , 0.5, 0.7, 0.8, 0.9],
], max_times=[1,0.3,1,1], fig_height=3)
plt.show()

### Trajectories with constant $F_N$

In [ ]:
data = np.load(base_path + "constant_Fn__m_L.npz", allow_pickle=True)
draw_trajs(data, [5, 6], plot_manipulator=False, progs=[0, 0.2,0.7,0.85,0.9,0.95],fig_height=2.5)
plt.show()

### Trajectories with offline trajectory

In [ ]:
data = np.load(base_path + "offline__m_L.npz", allow_pickle=True)
draw_trajs(data, [6, 7], plot_manipulator=True, progs=[
    list(np.linspace(0,1,5)),
    list(np.linspace(0,0.4,5)),
],fig_height=2.5, max_times=[1,0.4])
plt.show()

## Coefficient of friction

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__mu.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__mu.npz", allow_pickle=True),
    np.load(base_path + "offline__mu.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant", "Offline trajectory"]
colors = [measured_color, constant_color, offline_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.plot(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Coefficient of friction")
plt.ylabel("Angle about joint (radians)")
plt.gcf().savefig("mu_sweep.pgf", backend="pgf")
plt.show()

In [ ]:
data = np.load(base_path + "constant_Fn__mu.npz", allow_pickle=True)
draw_trajs(data, [3,4], plot_manipulator=True, fig_height=2.5,plot_pedestal=False)
plt.show()

In [ ]:
data = np.load(base_path + "offline__mu.npz", allow_pickle=True)
draw_trajs(data, [8, 9], plot_manipulator=True, fig_height=2.5)
plt.show()

# Control parameters

## Feedforward $F_N$

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__ff_FN.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__ff_FN.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant"]
colors = [measured_color, constant_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    plt.plot(x_axis, y_axis, '-', label=label, zorder=i, color=color)
#     plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
#     plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Additional $F_n$ (N)")
plt.ylabel("Angle about joint (radians)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [ ]:
data = np.load(base_path + "constant_Fn__ff_FN.npz", allow_pickle=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, Fn_noise) in enumerate(zip(data['out_data'], data['sweep_vars'].flatten())):
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Additional $F_n$ (N)")
plt.show()

In [ ]:
data = np.load(base_path + "constant_Fn__ff_FN.npz", allow_pickle=True)
idxs = [2, -6, -1]
Fns = data["sweep_vars"][idxs]
datas = data["out_data"][idxs]
sim_args = data["sim_args"][0]


fig, axs, cb = draw_trajs(data, idxs=idxs, progs=[
    [0, 0.2, 1],
    [0,0.04, 0.4,1],
    [0, 0.2, 0.4, 1]
])

for Fn, ax in zip(Fns, axs):
    ax.set_title("$F_N$ = " + str(Fn[0]))
plt.suptitle("System behavior with constant feedforward $F_N$")
plt.show()

## Impedance

In [ ]:
datas = [
    np.load(base_path + "offline__impedance.npz", allow_pickle=True),
    np.load(base_path + "measured_Fn__impedance.npz", allow_pickle=True),
]
labels = ["Offline", "Measured"]
colors = [offline_color, measured_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.semilogx(x_axis, y_axis, '--', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.xlabel("Additional $F_n$ (N)")
plt.ylabel("Angle about joint (radians)")
plt.show()

In [ ]:
data["exit_messages"]

In [ ]:
data["sweep_vars"]

# Noise
## $F_n$

In [ ]:
data = np.load(base_path + "measured_Fn__Fn_noise.npz", allow_pickle=True)

In [ ]:
num_samples = data["num_samples"]
num_noises = int(data["sweep_vars"].shape[0]/num_samples)
noises = data["sweep_vars"][::num_samples,0]
raw_y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
raw_succ_idxs = data["successes"].flatten()
raw_fail_idxs = np.logical_not(raw_succ_idxs)
portion_success = []

for i, noise in enumerate(noises):
    data_for_noise = raw_y_axis[i*num_samples:(i+1)*num_samples]
    mean = np.mean(data_for_noise)
    stddev = np.std(data_for_noise, ddof=1)
    plt.plot(np.ones(num_samples)*noise, data_for_noise, ' o', markersize=4, color=measured_color)
    plt.plot(noise, mean, ' x', markersize=5, color=measured_color)
    plt.plot([noise, noise], [mean-2*stddev,mean+2*stddev], '-', color=measured_color)
    
    succ_idxs = raw_succ_idxs[i*num_samples:(i+1)*num_samples]
    fail_idxs = raw_fail_idxs[i*num_samples:(i+1)*num_samples]
    portion_success.append(np.count_nonzero(succ_idxs)/num_samples)
plt.xlabel("Noise")
plt.ylabel("Angle about joint (radians)")
plt.show()

In [ ]:
portion_success       

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, Fn_noise) in enumerate(zip(
    data['out_data'][::data["num_samples"]], data['sweep_vars'].flatten()[::data["num_samples"]])):
    print(Fn_noise)
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("$F_N$ noise")
plt.gcf().savefig("Fn_noise_plot.pgf", backend="pgf")
plt.show()

In [ ]:
plt.figure()
plt.plot(data["out_data"][0]["times"], data["out_data"][0]["max_F_ONs"])
plt.show()

## Link noise

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__link_noise.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__link_noise.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant"]
colors = [measured_color, constant_color]
fig, axs = plt.subplots(len(datas), 1, figsize=(6,4), constrained_layout=True, sharex=True, sharey=True)
if len(datas) == 1:
    axs = [axs]
portion_successes = []

for data, label, color, ax in zip(datas, labels, colors, axs):
    num_samples = data["num_samples"]
    num_noises = int(data["sweep_vars"].shape[0]/num_samples)
    noises = data["sweep_vars"][::num_samples,0]
    raw_y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    raw_succ_idxs = data["successes"].flatten()
    raw_fail_idxs = np.logical_not(raw_succ_idxs)
    means = []
    portion_success = []
    
    for i, noise in enumerate(noises):
        # print(noise)
        data_for_noise = raw_y_axis[i*num_samples:(i+1)*num_samples]
        mean = np.mean(data_for_noise)
        means.append(mean)
        stddev = np.std(data_for_noise, ddof=1)
        ax.semilogx(
            [noise, noise], np.array([mean-2*stddev,mean+2*stddev])/success_angle, '-', color=color)
        
        x_axis = np.ones(num_samples)*noise
        y_axis = data_for_noise
        succ_idxs = raw_succ_idxs[i*num_samples:(i+1)*num_samples]
        fail_idxs = raw_fail_idxs[i*num_samples:(i+1)*num_samples]
        # print(succ_idxs)
        # print(np.count_nonzero(succ_idxs))
        
        portion_success.append(np.count_nonzero(succ_idxs)/num_samples)
        
        ax.scatter(x_axis[succ_idxs], y_axis[succ_idxs]/success_angle, color=color, zorder=i, alpha=0.1)
        ax.scatter(x_axis[fail_idxs], y_axis[fail_idxs]/success_angle, facecolors='none',
                   color=color, zorder=i, alpha=0.5)
    portion_successes.append(portion_success)
        
    ax.set_title(label)
    ax.semilogx(noises, np.array(means)/success_angle, '--x', markersize=10, color=color)
ax.set_xlabel("Link noise standard deviation")
ax.set_ylabel("Angle about joint (radians)")
plt.ylim(0,1.2)
plt.show()

In [ ]:
portion_successes

In [ ]:
datas = [
    np.load(base_path + "measured_Fn__link_noise.npz", allow_pickle=True),
    np.load(base_path + "constant_Fn__link_noise.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant"]
colors = [measured_color, constant_color]
fig, ax = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True, sharex=True, sharey=True)
portion_successes = []

for data, label, color in zip(datas, labels, colors):
    num_samples = data["num_samples"]
    num_noises = int(data["sweep_vars"].shape[0]/num_samples)
    noises = data["sweep_vars"][::num_samples,0]
    # raw_y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    raw_succ_idxs = data["successes"].flatten()
    raw_fail_idxs = np.logical_not(raw_succ_idxs)
    means = []
    portion_success = []
    
    for i in range(len(noises)):
        succ_idxs = raw_succ_idxs[i*num_samples:(i+1)*num_samples]
        fail_idxs = raw_fail_idxs[i*num_samples:(i+1)*num_samples]
        
        portion_success.append(np.count_nonzero(succ_idxs)/num_samples)
    portion_successes.append(portion_success)
    ax.semilogx(noises, portion_success, color=color, zorder=i)
        
    ax.set_title(label)
ax.set_xlabel("Link noise standard deviation")
ax.set_ylabel("Angle about joint (radians)")
plt.ylim(0,1.2)
plt.show()